In [ ]:
%matplotlib nbagg
import matplotlib as mpl
import matplotlib.pylab as plt
import numpy as np
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord

mpl.rcParams['mathtext.fontset']=r'custom'


sto2_plot_path = './Data/processed/'
odfile = sto2_plot_path+'eta_carinae_obs_positions_3801-3995_all_v3.npz'

idat = np.load(odfile)
print(idat.files)
gl = idat['gl']   
gb = idat['gb']   
gl0 = idat['gl0']   
gb0 = idat['gb0']   
gl1 = idat['gl1']   
gb1 = idat['gb1']   
gl2 = idat['gl2']   
gb2 = idat['gb2']   
gl3 = idat['gl3']   
gb3 = idat['gb3']   
gl4 = idat['gl4']   
gb4 = idat['gb4']   
gl5 = idat['gl5']   
gb5 = idat['gb5']   
gl6 = idat['gl6']   
gb6 = idat['gb6']   
gl7 = idat['gl7']   
gb7 = idat['gb7']   
gl8 = idat['gl8']   
gb8 = idat['gb8']   
gl9 = idat['gl9']   
gb9 = idat['gb9']   
scan0 = idat['scan0']
obsid0 = idat['obsid0']


fig = plt.figure(figsize=(10,6))
sp1 = plt.subplot()

pl1 = plt.plot(gl,gb,'+')

xrange = [287.20,287.455]
yrange = [-0.725,-0.480]
plt.xlim(xrange)
plt.ylim(yrange)


dl = np.abs(np.diff(gl0)*3600.)
esel = np.where((dl>10)&(dl<20.))
print('mean diff l: %.3f arcsec'%(np.mean(dl[esel])))
ldelt = np.mean(dl[esel])
print('extent l:  %.3f deg  %.3f deg'%(np.nanmin(gl), np.nanmax(gl)))
lext = np.abs(np.nanmin(gl)-np.nanmax(gl))
nl = lext / (np.mean(dl[esel])/3600.)
print('number of squares: ', np.ceil(nl))
print()


sel = np.where(gl<287.207)

dd = np.diff(gb)*3600
dsel = np.where((dd>10.)&(dd<18.))
bdelt = np.mean(dd[dsel])
print('mean diff b: %.3f arcsec'%(bdelt))
print('extent b:  %.3f deg  %.3f deg  %.3f'%(np.nanmin(gb), np.nanmax(gb), np.abs(np.nanmin(gb)-np.nanmax(gb))))
bext = np.abs(np.nanmin(gb)-np.nanmax(gb))
nb = bext / (np.mean(dd[dsel])/3600.)
print('number of squares: ', np.ceil(nb))
print()


# create the underlying coordinate grid
l0 = np.nanmin(gl)
lm = np.nanmax(gl) + ldelt/3600.
lgrid = np.flip(np.arange(l0, lm, ldelt/3600.))

print(lgrid.shape, lgrid.min(), lgrid.max())
print(lgrid)
for lg in lgrid:
    plt.plot([lg - ldelt/7200.,lg - ldelt/7200.], yrange, 'b')

b0 = np.nanmin(gb)
bm = np.nanmax(gb) + bdelt/3600.
bgrid = np.arange(b0, bm, bdelt/3600)
print(bgrid.shape, bgrid.min(), bgrid.max())
for bg in bgrid:
    plt.plot(xrange, [bg - bdelt/7200,bg - bdelt/7200], 'r')

    epsi_l = ldelt/7200.
epsi_b = bdelt/7200.
nx = 51
ny = 63
nn_grid = np.zeros((ny,nx))
for i in range(ny-1):
    for j in range(nx-1):
        args = np.argwhere((np.abs(gl-lgrid[j])<epsi_l)&(np.abs(gb-bgrid[i])<epsi_b))
        nn_grid[i,j] = args.size
        if nn_grid[i,j]>0: plt.text(lgrid[j],bgrid[i],'%i'%(nn_grid[i,j]))

            

In [ ]:
#
#
#     Note: this part takes a long time to finish !!!
#
#



%matplotlib nbagg
import matplotlib.pyplot as plt
import os
import sys
from scipy import sparse
from scipy.sparse.linalg import spsolve
from ALSFitter import *
import numpy as np
plt.rc("font", size=5)

ifile = './Data/processed/EtaCar_map1_0.7vp36_CII_2_3801_3993_v2_20200519.npz'

# detect the line reducced
if 'NII_1' in ifile: 
    lin = 0
    clin = 'NII_1'
if 'NII_2' in ifile: 
    lin = 1
    clin = 'NII_2'
if 'CII_2' in ifile: 
    lin = 2
    clin = 'CII_2'

ifp = ifile.split('_')
mvers = '_'+ifp[2]+'_'+ifp[3]+'_'+ifp[4]+'_'+(ifp[-1].split('.'))[0]
print('mvers: ', mvers)

dat = np.load(ifile)
sdat = dat['sdata']

# see what is available
skeys = sdat.dtype.names
print('Data file variables: ', skeys)


#full data arrays
velo = np.squeeze(sdat['vv'])
spec = np.squeeze(sdat['spec'])
sgl = np.squeeze(sdat['gl'])
sgb = np.squeeze(sdat['gb'])
print(velo.shape, spec.shape)

# the resampled data
nrpx = int(np.min(np.squeeze(sdat['nrpx'])))
spr = np.squeeze(sdat['spr'])[:,0:nrpx]
vvr = np.squeeze(sdat['vvr'])[:,0:nrpx]
bspr = np.squeeze(sdat['bspr'])[:,0:nrpx]
dspr = np.squeeze(sdat['dspr'])[:,0:nrpx]
scan = np.squeeze(sdat['scan'])
obsid = np.squeeze(sdat['sobsid'])

def resampleElements(aa1, aa2, nn):
    sh1 = aa1.shape
    sh2 = aa2.shape
    rdd1 = np.nanmean(aa1[:,0:sh1[1]//nn*nn].reshape((sh1[0],sh1[1]//nn,nn)), axis=2)
    rdd2 = np.nanmean(aa2[:,0:sh2[1]//nn*nn].reshape((sh2[0],sh2[1]//nn,nn)), axis=2)
    return rdd1, rdd2

# resample data and clip edges
vv, sp = resampleElements(velo, spec, 4)
vv = vv[:,15:-15]
sp = sp[:,15:-15]
print('full data sets: ', vv.shape, sp.shape)
print('coordinate data sets: ', sgl.shape, sgb.shape, gl.shape, gb.shape)
print('reduced data sets: ', spr.shape, vvr.shape, bspr.shape)
print('nrpx, #scans, #obsids: ', nrpx, scan.shape, obsid.shape)

xlim = [-70,50.]
ylim = [ -9.,33.]
nx=51
ny=63
strow = 0
stcol = 0
nrows = ny - strow - 1
ncols = nx - stcol - 1
print('creating subplots')

fig, axes2d = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(float(nx),float(ny)))
#fig, axes2d = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(40.,40.))
xx = np.array(np.arange(-100,70,1))
yy = np.random.random(xx.size)*5.
print('starting plotting loop')
fig.subplots_adjust(wspace=0.0, hspace=0.0, top=0.99, right=0.99)
#plt.setp(axes2d.flat, aspect=1.0, adjustable='box-forced')
for i, row in enumerate(axes2d):
    print(i)
    for j, cell in enumerate(row):
        ri = nrows-i-strow-1
        cj = j + stcol 
        args = np.argwhere((np.abs(sgl-lgrid[cj])<epsi_l)&(np.abs(sgb-bgrid[ri])<epsi_b))
        nn_grid[ri,cj] = args.size
        if nn_grid[ri,cj]>0: 
            #cell.text(xlim[0]+3, 26, '%.3f %.3f %i %i %i'%(lgrid[cj],bgrid[ri],nn_grid[ri,cj],cj,ri))
            cell.text(xlim[0]+3, 26, '%.3f %.3f'%(lgrid[cj],bgrid[ri]))
            cell.grid()
            cell.tick_params('x', length=5, width=1, tickdir='in')
            #cell.text(xlim[0]+3, 18, '%i %i '%(scan[args[0]],obsid[args[0]]))
            for k,ag in enumerate(args):
                cell.text(xlim[0]+3, 22-k*3, '%i %i '%(scan[args[k]],obsid[args[k]]))
                cell.plot(np.squeeze(vvr[ag,:]), np.squeeze(spr[ag,:]) - np.squeeze(dspr[ag,:]), drawstyle='steps-mid', alpha=0.95-k*0.3)

plt.xlim(xlim)
plt.ylim(ylim)
# plt.tight_layout()

print('Done loop, creating pdf file. Be patient...')


ofile = 'EtaCar_spec_map_vt_%s.pdf'%(mvers)
pdf_file3 = os.path.join('./Data/processed/',ofile)
plt.savefig(pdf_file3,format='PDF',orientation='landscape')
print(pdf_file3)

